# BLANK FEDS PEC (Polygon Evaluation and Comparison) Outline

### Summary

This notebook is a blank starting point for users to use the PEC implementation. Refer to the `demo` directory for demonstrations on using the outline

The FEDS input and reference input can use either a pre-defined setting or can accept (valid) user-written paths to shape files. 
- For Input all settings, please see **section INPUT SETTINGS of README.md**  
- Output settings can also be modified to control if calculations are printed, plotted, and/or stored into a file (with location and file type of choice); for more detail on outputs please see **section OUTPUT SETTINGS of README.md**

#### Debugging
Should users encounter abnormal errors, users are encouraged to restart the kernel of the notebook and re-run blocks. 

#### Notes
- FEDS perimeters are plotted as <span style='color:red'> **red** </span>
- Reference perimeters are plotted as <span style='color:#B8860B'> **transparent hatch gold** </span>
- Users may need to shorten query time intervals, as the FEDS API is limited to 9000 results per query. Should the limit be exceeded an error will be shown

### User Inputs for Comparison: time, bbox, FEDS set, reference set

Note for inputs:

FEDS Input: you must specify a valid title (e.g. firenrt) and collection (e.g. public.eis_fire_lf_perimeter_archive) according to documentation. Please see **section INPUT SETTINGS of README.md** for valid titles and corresponding collections

Reference Input: See **section INPUT SETTINGS of README.md** for valid titles and corresponding collections

In [ ]:
import sys
sys.path.append('../') # local path append

import Utilities
import Input_FEDS
import Input_Reference
import Output_Calculation

from Input_FEDS import InputFEDS
from Input_Reference import InputReference
from Output_Calculation import OutputCalculation
from Utilities import *

In [ ]:
# this is the ONLY section users are intended to modify!
# inputs for searching FEDS and nifc sets

# START TIME
year_start = ... # e.g. 2020
month_start = ... # e.g. 7 
day_start = ... # e.g. 1
hour_start = ... # e.g. 0
minute_start = ... # e.g. 0
second_start = ... # e.g. 0
tz_offset_hours_start = ... # e.g. 0
tz_offset_minutes_start = ... # e.g. 0
utc_offset_start = ... # e.g. '00:00'

# END TIME
year_stop = ... # e.g. 2020
month_stop = ... # e.g. 8
day_stop = ... # e.g. 30
hour_stop = ... # e.g. 0
minute_stop = ... # e.g. 0
second_stop = ... # e.g. 0
tz_offset_hours_stop = ... # e.g. 0
tz_offset_minutes_stop = ... # e.g. 0
utc_offset_stop = ... # e.g. '00:00'

# CRS 
crs = ... # e.g. 3857

# BBOX FOR SEARCH - [lon, lat, lon, lat]
search_bbox = ... # e.g. ["-125.0", "24.396308", "-66.93457", "49.384358"] 

# DAY SEARCH RANGE- acceptable distance warning to search from feds -> reference (e.g. if reference polygon is 8 days away, output a warning)
# note by default, any results not of the same year are eliminated
day_search_range = ... # e.g. 7 

# FEDS INPUT SETTINGS
feds_title = ... # e.g.  "firenrt"
feds_collection = ... # e.g. "public.eis_fire_lf_perimeter_archive"
feds_access_type = ... # e.g. "api"
feds_limit = ... # e.g. 9000 # amount of features to consider for FEDS API access; warning appears if it misses any entries
feds_filter = ... # e.g. False
feds_apply_finalfire = ... # e.g. True

# REFERENCE INPUT SETTINGS 
ref_title = ... # e.g. "Downloaded_InterAgencyFirePerimeterHistory_All_Years_View"
ref_control_type = ... # e.g. "defined" # or "custom"
ref_custom_url = ... # e.g. "none"
ref_custom_read_type = ... # e.g. "none"
ref_filter = ... # e.g. False # False or a valid query

# OUTPUT SETTINGS - User discretion advised for path functionality
print_on = ... # e.g. True
plot_on = ... # e.g. True
name_for_output_file = ... # e.g. "test_run"
output_format = ... # e.g. "csv"
user_path = ... # e.g. "/projects/my-public-bucket/VEDA-PEC/results"
output_maap_url = ... # e.g. f"{user_path}/{name_for_output_file}.{output_format}"

### Argument Processing: format and check to report any issues
- Note: upon instance instantiation we can also get reports back of invalidity -> any warnings/errors should be addressed

In [ ]:

# start date formatting
search_start = Utilities.format_datetime(year_start, 
                                         month_start, 
                                         day_start, 
                                         hour_start, 
                                         minute_start, 
                                         second_start, 
                                         tz_offset_hours_start, 
                                         tz_offset_minutes_start,
                                         utc_offset_start)
# stop date formatting
search_stop = Utilities.format_datetime(year_stop, 
                                        month_stop, 
                                        day_stop, 
                                        hour_stop, 
                                        minute_stop, 
                                        second_stop, 
                                        tz_offset_hours_stop, 
                                        tz_offset_minutes_stop,
                                        utc_offset_stop)

# bound check the bbox
assert Utilities.check_bbox(search_bbox), f"ERR: passed bbox {search_bbox} is not valid; check bounds"
assert  Utilities.check_crs(crs), f"ERR: invalid crs provided {crs}; please enter valid ESPG CRS number"

### Using Inputs: Instantiate Inputs + Calculation Output

With our inputs, we are ready to start; run this to get the program to fetch the FEDS candidates in the provided time/location window, along with an intersecting reference in the specific day range

In [ ]:
# create instances to track properties
print('You may see an ERROR 1 occur; ignore this unless block throws actual exception...')

feds_firenrt = InputFEDS(
                 feds_title, 
                 feds_collection, 
                 search_start,
                 search_stop,
                 search_bbox,
                 crs,
                 feds_access_type,
                 feds_limit,
                 feds_filter,
                 feds_apply_finalfire
                )

nifc_search = InputReference( 
                 search_start,
                 search_stop,
                 search_bbox,
                 crs,
                 ref_title,
                 ref_control_type,
                 ref_custom_url,
                 ref_custom_read_type,
                 ref_filter,
                )

my_output = OutputCalculation(
                feds_firenrt,
                nifc_search,
                output_format, 
                output_maap_url,
                day_search_range,
                print_on,
                plot_on
                )

### Tif Analysis on Symmetric Difference Matches (Exceeding Areas of FEDS over Reference)

This function built into the Output Class demonstrates research application uses of FEDS-PEC

Given the output matches, give a tif path, calculations of interest, and optional time (in days) limit. This will return associated TIF values for the symmetric difference of FEDS over the reference (FEDS - reference)

In [ ]:
# tif_path - replace with your own path
tif_path = ... # e.g. "/projects/my-public-bucket/tif_files/slopeLF/LC20_SlpD_220.tif"

# array of values returned as get_res
get_res = my_output.tif_analysis(tif_path, "MEDIAN", 100)

### Export Polygons

Provide a path along with polygons, and then call on the `export_polygons` method for your OutputCalculation object

In [ ]:
# name your file
shp_file_name = ... # e.g. "feds_used"
# resuses the same dir as the csv export 
output_shp_path = f"{user_path}/{shp_file_name}.shp"

my_output.export_polygons(feds_firenrt.polygons, output_shp_path)